In [1]:
import os
from tracktour import Tracker, get_im_centers

In [2]:
DATA_ROOT = '/home/ddon0001/PhD/data/cell_tracking_challenge/ST/'
DS_NAME = 'Fluo-N2DL-HeLa/'
SEQ = '02_ST'
GT_PATH = os.path.join(DATA_ROOT, DS_NAME, '02_GT/TRA/')
SEG_PATH = os.path.join(DATA_ROOT, DS_NAME, SEQ, 'SEG/')

In [3]:
seg_labels, detections, min_t, max_t, corners = get_im_centers(SEG_PATH)
print(f"Loaded {seg_labels.shape[0]} frames of shape {seg_labels.shape[1:]}.")
print(f"Extracted {len(detections)} detections:")
print(detections.head())

Extracting Centroids: 100%|██████████| 92/92 [00:02<00:00, 31.01it/s]

Loaded 92 frames of shape (700, 1100).
Extracted 25378 detections:
   t           y           x  label
0  0  105.033501   17.544389      1
1  0  106.990672  117.902985     20
2  0   73.033403  143.325678     24
3  0  108.000000  171.669355     28
4  0  141.251956  138.192488     32


In [4]:
tracker = Tracker(
    im_shape = corners[1],
    k_neighbours=10
)
tracked = tracker.solve(
    detections,
    frame_key="t",
    location_keys=("y", "x"),
    value_key="label"
)
print(tracked.tracked_detections.head())
print(tracked.tracked_edges.head())

Computing candidate edges: 100%|██████████| 91/91 [00:00<00:00, 1279.92it/s]


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-24
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11850H @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 75774 rows, 325923 columns and 1248939 nonzeros
Model fingerprint: 0xf07cfc27
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-03, 3e+02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3 rows and 2 columns
Presolve time: 0.43s
Presolved: 75771 rows, 325921 columns, 1248909 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.38s

Barrier performed 0 iterations in 0.91 seconds (0.91 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective  

In [5]:
# convert solution to networkx graph
sol_graph = tracked.as_nx_digraph()

# convert to CTC output
from tracktour import get_ctc_output
seg, tdf = get_ctc_output(seg_labels, sol_graph, frame_key='t', value_key='label')


In [6]:
# evaluate using `traccuracy` - note `traccuracy` should be installed separately!

from traccuracy import TrackingGraph
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatcher
from traccuracy.metrics import CTCMetrics

comp_graph = TrackingGraph(
    sol_graph, 
    label_key="label", 
    location_keys=["y", "x"], 
    segmentation=seg_labels
)

gt_graph = load_ctc_data(GT_PATH)
matcher = CTCMatcher()
matched = matcher.compute_mapping(gt_graph, comp_graph)
results = CTCMetrics().compute(matched)

print(results.results)

ModuleNotFoundError: No module named 'traccuracy'